In [2]:
# loading libraries
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

Using TensorFlow backend.


In [30]:
# Load data set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [36]:
print("x_train size:", x_train.shape)
print("y_train size:", y_train.shape)

print("\nx_test size:", x_test.shape)
print("y_test size:", y_test.shape)

# The x_train is the training input dataset with 50,000 images of 32*32*3 dimensions
# The y_train is the training output dataset which has labels for the images

x_train size: (50000, 32, 32, 3)
y_train size: (50000, 1)

x_test size: (10000, 32, 32, 3)
y_test size: (10000, 1)


In [37]:
# CIFAR10 has ten types of images labeled from 0 to 9. We only care about birds, which are labeled as class #2.
# So we'll change the Y values. Instead of each class being labeled from 0 to 9, we'll set it to True
# if it's a bird and False if it's not a bird.

y_train = (y_train == 2).astype(int)
y_test = (y_test == 2).astype(int)

In [38]:
# Normalize image data (pixel values from 0 to 255) to the 0-to-1 range
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [44]:
# Create a model and add layers
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(32, 32, 3), activation="relu"))
# The 32 parameter tells Keras to create 32 convolutional filters. 
# The (3, 3)parameter means that each convolutional filter will be three pixels wide and three pixels tall.
# Padding parameter tells Keras what to do at the edges of the image. 'same' means that Keras will add padding 
# at the edges of the images and valid means that Keras won’t add any padding (which is the default).
# Input_shape parameter tells Keras size of the input data i.e 32x32 pixels with 3 color channels (RGB).
# Rectified linear unit or relu is the activation function
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
# MaxPooling layer to downsample the data, A (2, 2) pool size means that the largest value in each 2x2 block of 
# data will be kept. This will downsample our data to 25 percent of its original size.
model.add(BatchNormalization())
# BatchNormalization layer will continually normalize the data as it passes between layers. 
# This just helps the model train a little faster.
model.add(Dropout(0.25))
# Dropout layer is to help prevent overfitting.

# Adding another group of convolutional layers, max pooling, batch normalization, and dropout.
model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

# Flatten() call is required in Keras whenever you transition from convolutional layers to Dense layers. 
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid")) # this is the output layer

# The output layer uses a sigmoid activation instead of a relu activation. The sigmoid function always produces 
# a value between 0 and 1. That’s perfect for the output layer since we want to output a probability score.

In [45]:
# Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer="adam",
    metrics=['accuracy']
)

In [46]:
# Train the model
model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=1, # running on local machine so using epoch=1, ideally 100-200
    validation_data=(x_test, y_test),
    shuffle=True
)

# Save the trained model to a file so we can use it to make predictions later
model.save("bird_model.h5")

Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 164s 3ms/step - loss: 6.7485e-04 - acc: 0.9997 - val_loss: 1.0482e-07 - val_acc: 1.0000
